# SciPy

SciPy provides algorithms for optimization, integration, interpolation, eigenvalue problems, algebraic equations, differential equations, statistics and many other classes of problems. It is a collection of mathematical algorithms and convenience functions built on `NumPy` . SciPy adds significant power to Python by providing the user with high-level commands and classes for manipulating and visualizing data. As always, visit its [website](https://scipy.org/) to find more information about the package. 

## Subpackages

SciPy is organized into subpackages covering different scientific computing domains. The most relevant for us are:

1. Optimize. Contains optimization and root-finding routines. 
2. Integrate. Integration and differential equaiton solver. 
3. Interpolate. Interpolation and smoothing splines. 

## Optimize

The `scipy.optimize` package provides several commonly used optimization algorithms. Here we will revise the main functions for constrained and unconstrained optimization. Visit [here](https://docs.scipy.org/doc/scipy/tutorial/optimize.html) to have access to the full list. 

### scipy.optimize.minimize
Minimization of scalar function of one or more variables. See documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html.). As an example, we will now minimize the Rosenbrok function: 
$$f(x) = \sum_{i=1}^{N-1}100(x_{i+1}-x_{i}^2) + (1-x_{i})^2$$
The minimum is $0$, achieved when $x_i=1$.



In [1]:
import numpy as np
from scipy.optimize import minimize

def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])  # Initial Guess! 
res = minimize(rosen, x0)

# We can access the resuts of the optimization problem
print(res.x)
print(res)


[0.99999925 0.99999852 0.99999706 0.99999416 0.99998833]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 4.581838024995112e-11
        x: [ 1.000e+00  1.000e+00  1.000e+00  1.000e+00  1.000e+00]
      nit: 25
      jac: [-5.789e-06 -2.823e-06 -2.798e-06 -7.643e-06  5.855e-06]
 hess_inv: [[ 7.585e-03  1.243e-02 ...  4.613e-02  9.217e-02]
            [ 1.243e-02  2.481e-02 ...  9.296e-02  1.856e-01]
            ...
            [ 4.613e-02  9.296e-02 ...  3.737e-01  7.459e-01]
            [ 9.217e-02  1.856e-01 ...  7.459e-01  1.494e+00]]
     nfev: 180
     njev: 30


If we want, we can choose the optmization algorithm. For example, let's try the Nelder-Mead Simplex algorithm

In [3]:
res = minimize(rosen, x0, method='nelder-mead')
print(res.x)

[0.99910115 0.99820923 0.99646346 0.99297555 0.98600385]


### Constrained Optimization 

The minimize function provides algorithms for constrained minimization, namely 'trust-constr' , 'SLSQP' and 'COBYLA'. They require the constraints to be defined using slightly different structures. The method 'trust-constr' requires the constraints to be defined as a sequence of objects LinearConstraint and NonlinearConstraint. Methods 'SLSQP' and 'COBYLA', on the other hand, require constraints to be defined as a sequence of dictionaries, with keys type, fun and jac. 

Therefore, the way we define the constraints will be sensitive to the optimization algorithm that we are using. 

As an example let us consider the constrained minimization of the Rosenbrock function:

$$ min_{x0,x1} 100(x_1 - x_0^2) + (1-x_0)^2 $$
subject to:
$$x_0 + 2x_1 \leq 1 \\ 
x_0^2 + x_1 \leq 1 \\
x_0^2 - x_1 \leq 1 \\
2x_0 + x_1 = 1 \\
0 \leq x_0 \leq 1 \\
-0.5 \leq x_1 \leq 2 $$

This optimization problem has the unique solution $[x_0,x_1] = [0.4149,0.1701]$, for which only the first and fourth constraints are active.  To solve this problem we will be using the Trust-Region Constrained Algorithm.  We will now proceed as following: 
1. Define the Bounds. 
2. Define the Linear Constraints. 
3. Define the NonLinear Constraints. 
4. Solving the Optimization Problem. 

#### Defining Bounds

In [4]:
from scipy.optimize import Bounds

bounds = Bounds([0,-0.5],[1,2])

#### Defining Linear Constraints:



In [5]:
from scipy.optimize import LinearConstraint
linear_constraint = LinearConstraint([[1, 2], [2, 1]], [-np.inf, 1], [1, 1])

#### Defining Nonlinear Constraints:

In [6]:
def cons_f(x):
    return [x[0]**2 + x[1], x[0]**2 - x[1]]
def cons_J(x):
    return [[2*x[0], 1], [2*x[0], -1]]
def cons_H(x, v):
    return v[0]*np.array([[2, 0], [0, 0]]) + v[1]*np.array([[2, 0], [0, 0]])
from scipy.optimize import NonlinearConstraint
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 1, jac=cons_J, hess=cons_H)

#### Solving the optimization problem


In [8]:
x0 = np.array([0.5, 0])
res = minimize(rosen, x0, method='trust-constr',
               constraints=[linear_constraint, nonlinear_constraint],
               options={'verbose': 1}, bounds=bounds)

print(res.x)

`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 24, CG iterations: 7, optimality: 4.44e-09, constraint violation: 0.00e+00, execution time: 0.029 s.
[0.41494531 0.17010937]


#### Exercice. Solve the following optimization problem: 

$$min_{x1,x2} x_1^2 + x_2^2 $$
subject to
$$ 1 \leq x_1 \leq 4 \\ 
-3 \leq 2 x_2 \leq -2$$

In [10]:
def objective_function(x):

    return x[0]**2 + x[1]**2

bounds = Bounds([1,-3],[4,-2])

x0 = [3,-1.5]

res = minimize(objective_function, x0, bounds=bounds)

print(res.x)

[ 1. -2.]


### Root Finder

The main function to find the root of a function is `fsolve()`. You can find the documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html).  Let's see it in action trying to find a solution to the following system of equations: 
$$ x_0\cos{x_1} = 4 \\ 
x_0x_1 - x_1 = 5$$

In [12]:
import numpy as np
from scipy.optimize import fsolve
def func(x):
    return [x[0] * np.cos(x[1]) - 4,
            x[1] * x[0] - x[1] - 5]
root = fsolve(func, [1, 1])  # inputs are the system of equations and the initial guess of their roots. 
root

array([6.50409711, 0.90841421])

## Integration

There are multiple ways to perform integration using the `scipy.integrate` library. You can find a revision in [here](https://www.geeksforgeeks.org/scipy-integration/). For the porpuse of this class we will just see a couple of examples extracted from the previous link. 

### Single variable integration

In [13]:
from scipy.integrate import quad 
  
def f(x): 
  return 3.0*x*x + 1.0
  
I, err = quad(f, 0, 1) 
print(I) # Integral
print(err) # Integration error

2.0
2.220446049250313e-14


### Double integration

In [14]:
from scipy.integrate import dblquad 
  
area = dblquad(lambda x, y: x*y, 0, 0.5,  
               lambda x: 0, lambda x: 1-2*x) 
  
print(area)

(0.010416666666666668, 4.101620128472366e-16)


### n-dimensional integral

In [15]:
from scipy.integrate import nquad 
  
  
def f(x, y, z): 
    return x*y*z 
  
  
I = nquad(f, [[0, 1], [0, 5], [0, 5]]) 
print(I) 

(78.12499999999999, 8.673617379884033e-13)


<div class="alert alert-block alert-info"> <b>TIP!</b> If you are trying to apply integration techniques to your coude, you might want to learn about Gaussian Quadratures. </div>

Visit the link [here](https://en.wikipedia.org/wiki/Gaussian_quadrature) for more information about it.